In [363]:
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from cleaning_data import *
from stats import *
from functions import *
from clean_data_testing import *
from utils import split_data, stratified_train_test_split, downsample_class_0
import datetime
import seaborn as sns
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [364]:
from functions import *

In [365]:
from clean_data_testing import *

In [366]:
DATA_PATH = '/Users/williamjallot/Desktop/ML/dataset'
x_train, x_test, y_train, train_ids, test_ids, labels =  load_csv_data(DATA_PATH, sub_sample=False)
labels.pop(0) 

'Id'

In [367]:
x_tr, x_te, y_tr, y_te = split_data(x_train, y_train, 0.8, seed= 42)

In [368]:
y_tr[y_tr == -1] = 0

In [369]:
def calculate_downsample_size(N_pos, N_neg, desired_percentage):
    # Calculate the required downsample size for the negative class
    downsample_size_neg = int((N_pos * (1 - desired_percentage)) / desired_percentage)
    
    # Ensure we do not downsample more than available negative samples
    downsample_size_neg = min(downsample_size_neg, N_neg)
    
    return downsample_size_neg

In [370]:
N_pos = len(np.where(y_tr == 1)[0])
N_neg = len(np.where(y_tr == 0)[0])

In [371]:
down_sampling_size = calculate_downsample_size(N_pos, N_neg, 0.25)

In [372]:
x_tr_ds, y_tr_ds = downsample_class_0(x_tr, y_tr, down_sampling_size)

In [373]:
x_tr_ds.shape

(92300, 321)

In [401]:
x_train_cleaned, features,median_and_most_probable_class  = clean_data_x(x_tr_ds, labels)

    
np.savetxt("/Users/williamjallot/Desktop/ML/dataset/x_trained.cleaned.csv", x_train_cleaned, delimiter=",")

In [402]:
x_te_cleaned = clean_data_testing(x_te, labels, features, median_and_most_probable_class)

In [411]:
max_iters = 100000
gamma = 0.01
batch_size = 32
lambda_ =  0.005
num_samples = x_train_cleaned.shape[0]



# Parameters for the Gaussian distribution
mean = 0    # Mean of the distribution
std_dev = 1 # Standard deviation of the distribution

tx = np.c_[np.ones(num_samples), x_train_cleaned]
w_initial = np.random.normal(loc=mean, scale=std_dev, size=tx.shape[1])


sgd_losses, sgd_ws = stochastic_gradient_descent(
y_tr_ds, tx, w_initial, batch_size, max_iters, gamma, lambda_
)


In [412]:
num_samples = x_te.shape[0]
tx_te = np.c_[np.ones(num_samples), x_te_cleaned]
y_predict = predict(y_te, tx_te, sgd_ws[-1])

In [413]:
accuracy(y_te, y_predict)

0.8256510277782011

In [414]:
f1_score(y_te, y_predict)

(0.26446387246078684, 0.34864406779661017)